In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_columns=None
pd.options.display.max_rows=None

## 1. Load Dataset - German Credit Dataset

In [2]:
#set path
path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data'
dataset = pd.read_csv(path, delimiter=' ', header=None)

In [3]:
#shape
dataset.shape

(1000, 21)

In [4]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,4,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,2,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,3,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,4,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,4,A124,53,A143,A153,2,A173,2,A191,A201,2


In [5]:
#list of column names
COL = [
    'Status_of_existing_checking_account', 
    'Duration_in_month',
    'Credit_history',
    'Purpose', 
    'Credit_amount', 
    'Savings_account_bonds', 
    'Present_employment_since', 
    'Installment_rate_in_percentage_of_disposable_income',
    'Personal_status_and_sex',
    'Other_debtors_guarantors',
    'Present_residence_since', 
    'Property', 
    'Age_in_years', 
    'Other_installment_plans', 
    'Housing',
    'Number_of_existing_credits_at_this_bank',
    'Job',
    'Number_of_people_being_liable_to_provide_maintenance_for',
    'Telephone',
    'foreign_worker',
    'Target'
]

In [11]:
dataset.columns = COL

In [12]:
dataset.head()

,Status_of_existing_checking_account,Duration_in_month,Credit_history,Purpose,Credit_amount,Savings_account_bonds,Present_employment_since,Installment_rate_in_percentage_of_disposable_income,Personal_status_and_sex,Other_debtors_guarantors,Present_residence_since,Property,Age_in_years,Other_installment_plans,Housing,Number_of_existing_credits_at_this_bank,Job,Number_of_people_being_liable_to_provide_maintenance_for,Telephone,foreign_worker,Target
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,4,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,2,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,3,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,4,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,4,A124,53,A143,A153,2,A173,2,A191,A201,2


Label

1 : Good (good credibility) → change to 0<br>
2 : Bad (bad credibility) → change to 1

In [13]:
# change label from 1/2 to 0/1
dataset['Target'] = dataset['Target'] - 1

## 2. Information Value

$$WOE = \ln \frac{\text{distribution of good}}{\text{distribution of bad}}$$

In [14]:
#categorical feature은 그 category 하나에 대해서 IV를 구한다
#numerical feature은 통상적으로 10개의 bin을 만든다

In [15]:
max_bin = 10  # 전체 데이터의 예측력에 해를 가하지 않는 한에서 구간을 테스트하였습니다.
def calc_iv(df, col, label, max_bin = max_bin):
    """IV helper function"""
    bin_df = df[[col, label]].copy()
    # Categorical column
    if bin_df[col].dtype == 'object':
        bin_df = bin_df.groupby(col)[label].agg(['count', 'sum'])
    # Numerical column
    else:
        bin_df.loc[:, 'bins'] = pd.qcut(bin_df[col].rank(method='first'), max_bin)
#         bin_df.loc[:, 'bins'] = pd.cut(bin_df[col], max_bin)
        bin_df = bin_df.groupby('bins')[label].agg(['count', 'sum'])
    
    bin_df.columns = ['total', 'abuse']
    bin_df['normal'] = bin_df['total'] - bin_df['abuse']
    bin_df['normal_dist'] = bin_df['normal'] / sum(bin_df['normal'])
    bin_df['abuse_dist'] = bin_df['abuse'] / sum(bin_df['abuse'])
    bin_df['woe'] = np.log(bin_df['normal_dist'] / bin_df['abuse_dist'])
    bin_df['iv'] = bin_df['woe'] * (bin_df['normal_dist'] - bin_df['abuse_dist'])
    
    bin_df.replace([np.inf, -np.inf], 0, inplace=True)
    bin_df = bin_df[bin_df['total'] > 0]
    iv_val = sum(filter(lambda x: x != float('inf'), bin_df['iv']))
    
    return bin_df, col, iv_val


In [16]:
ch_df, ch, ch_i_val = calc_iv(dataset,'Credit_history', 'Target')
ch_df

,total,abuse,normal,normal_dist,abuse_dist,woe,iv
Credit_history,,,,,,,
A30,40,25,15,0.021429,0.083333,-1.358123,0.084074
A31,49,28,21,0.030000,0.093333,-1.134980,0.071882
A32,530,169,361,0.515714,0.563333,-0.088319,0.004206
A33,88,28,60,0.085714,0.093333,-0.085158,0.000649
A34,293,50,243,0.347143,0.166667,0.733741,0.132423


In [19]:
print("information value : " + str(ch_i_val))

information value : 0.2932335473908263


In [20]:
dim_df, dim, dim_i_val = calc_iv(dataset,'Duration_in_month', 'Target')
dim_df

,total,abuse,normal,normal_dist,abuse_dist,woe,iv
bins,,,,,,,
"(0.999, 100.9]",100,11,89,0.127143,0.036667,1.243443,0.112502
"(100.9, 200.8]",100,20,80,0.114286,0.066667,0.538997,0.025667
"(200.8, 300.7]",100,27,73,0.104286,0.090000,0.147325,0.002105
"(300.7, 400.6]",100,25,75,0.107143,0.083333,0.251314,0.005984
"(400.6, 500.5]",100,26,74,0.105714,0.086667,0.198671,0.003784
"(500.5, 600.4]",100,38,62,0.088571,0.126667,-0.357750,0.013629
"(600.4, 700.3]",100,32,68,0.097143,0.106667,-0.093526,0.000891
"(700.3, 800.2]",100,31,69,0.098571,0.103333,-0.047179,0.000225
"(800.2, 900.1]",100,42,58,0.082857,0.140000,-0.524524,0.029973


In [21]:
dim_i_val

0.2605225223321392

In [26]:
# 함수를 만들어서 전체 iv를 살펴보자
col_iv = {}
for col in dataset.columns.tolist():
    if col == 'Target':
        continue
    _, col, iv = calc_iv(dataset, col, 'Target')
    col_iv[col] = iv

In [27]:
col_iv

{'Status_of_existing_checking_account': 0.6660115033513336,
 'Duration_in_month': 0.2605225223321392,
 'Credit_history': 0.2932335473908263,
 'Purpose': 0.16919506567307832,
 'Credit_amount': 0.11342803024552867,
 'Savings_account_bonds': 0.19600955690422672,
 'Present_employment_since': 0.086433631026641,
 'Installment_rate_in_percentage_of_disposable_income': 0.061554683786294126,
 'Personal_status_and_sex': 0.04467067763379073,
 'Other_debtors_guarantors': 0.032019322019485055,
 'Present_residence_since': 0.04874371881018562,
 'Property': 0.11263826240979674,
 'Age_in_years': 0.10267245670259074,
 'Other_installment_plans': 0.057614541955647885,
 'Housing': 0.08329343361549926,
 'Number_of_existing_credits_at_this_bank': 0.09779114631307396,
 'Job': 0.008762765707428294,
 'Number_of_people_being_liable_to_provide_maintenance_for': 0.03408883520785682,
 'Telephone': 0.0063776050286746735,
 'foreign_worker': 0.04387741201028899}

In [28]:
import operator
candidates = sorted(col_iv.items(), key=operator.itemgetter(1), reverse=True)
display(candidates)

[('Status_of_existing_checking_account', 0.6660115033513336),
 ('Credit_history', 0.2932335473908263),
 ('Duration_in_month', 0.2605225223321392),
 ('Savings_account_bonds', 0.19600955690422672),
 ('Purpose', 0.16919506567307832),
 ('Credit_amount', 0.11342803024552867),
 ('Property', 0.11263826240979674),
 ('Age_in_years', 0.10267245670259074),
 ('Number_of_existing_credits_at_this_bank', 0.09779114631307396),
 ('Present_employment_since', 0.086433631026641),
 ('Housing', 0.08329343361549926),
 ('Installment_rate_in_percentage_of_disposable_income', 0.061554683786294126),
 ('Other_installment_plans', 0.057614541955647885),
 ('Present_residence_since', 0.04874371881018562),
 ('Personal_status_and_sex', 0.04467067763379073),
 ('foreign_worker', 0.04387741201028899),
 ('Number_of_people_being_liable_to_provide_maintenance_for',
  0.03408883520785682),
 ('Other_debtors_guarantors', 0.032019322019485055),
 ('Job', 0.008762765707428294),
 ('Telephone', 0.0063776050286746735)]

In [31]:
# 분석하려는 feature의 갯수와 예측력의 trade-off를 조정하였습니다.
iv_cols = [key for key, iv in candidates if iv >= 0.044]
display(len(iv_cols))
display(sorted(iv_cols))

15

['Age_in_years',
 'Credit_amount',
 'Credit_history',
 'Duration_in_month',
 'Housing',
 'Installment_rate_in_percentage_of_disposable_income',
 'Number_of_existing_credits_at_this_bank',
 'Other_installment_plans',
 'Personal_status_and_sex',
 'Present_employment_since',
 'Present_residence_since',
 'Property',
 'Purpose',
 'Savings_account_bonds',
 'Status_of_existing_checking_account']

In [32]:
# 상수로 설정합니다.
IV_COL = iv_cols[:]

In [33]:
# Category column들은 one hot vectore로 변환합니다.
cate_features = {}
num_fetures = []
for col in IV_COL:
    if dataset[col].dtype == 'object':
        cate_features[col] = pd.get_dummies(dataset[col], prefix=col)
    else:
        num_fetures.append(col)

In [34]:
cate_features.keys()

dict_keys(['Status_of_existing_checking_account', 'Credit_history', 'Savings_account_bonds', 'Purpose', 'Property', 'Present_employment_since', 'Housing', 'Other_installment_plans', 'Personal_status_and_sex'])

In [35]:
cate_features['Status_of_existing_checking_account'].head()

,Status_of_existing_checking_account_A11,Status_of_existing_checking_account_A12,Status_of_existing_checking_account_A13,Status_of_existing_checking_account_A14
0,1,0,0,0
1,0,1,0,0
2,0,0,0,1
3,1,0,0,0
4,1,0,0,0


In [36]:
removed_features = []

In [37]:
for col, dummies in cate_features.items():
    dropped_col = dummies.columns[-1]
    removed_features.append(dropped_col)
    cate_features[col] = dummies.drop(dropped_col, axis=1)

In [38]:
removed_features

['Status_of_existing_checking_account_A14',
 'Credit_history_A34',
 'Savings_account_bonds_A65',
 'Purpose_A49',
 'Property_A124',
 'Present_employment_since_A75',
 'Housing_A153',
 'Other_installment_plans_A143',
 'Personal_status_and_sex_A94']

In [39]:
cate_features['Status_of_existing_checking_account'].head()

,Status_of_existing_checking_account_A11,Status_of_existing_checking_account_A12,Status_of_existing_checking_account_A13
0,1,0,0
1,0,1,0
2,0,0,0
3,1,0,0
4,1,0,0


In [40]:
final_dataset = dataset[num_fetures]

In [41]:
for col, df in cate_features.items():
    final_dataset = pd.concat([final_dataset, df], axis=1)

In [42]:
final_dataset.shape

(1000, 40)

In [43]:
final_dataset.head()

,Duration_in_month,Credit_amount,Age_in_years,Number_of_existing_credits_at_this_bank,Installment_rate_in_percentage_of_disposable_income,Present_residence_since,Status_of_existing_checking_account_A11,Status_of_existing_checking_account_A12,Status_of_existing_checking_account_A13,Credit_history_A30,Credit_history_A31,Credit_history_A32,Credit_history_A33,Savings_account_bonds_A61,Savings_account_bonds_A62,Savings_account_bonds_A63,Savings_account_bonds_A64,Purpose_A40,Purpose_A41,Purpose_A410,Purpose_A42,Purpose_A43,Purpose_A44,Purpose_A45,Purpose_A46,Purpose_A48,Property_A121,Property_A122,Property_A123,Present_employment_since_A71,Present_employment_since_A72,Present_employment_since_A73,Present_employment_since_A74,Housing_A151,Housing_A152,Other_installment_plans_A141,Other_installment_plans_A142,Personal_status_and_sex_A91,Personal_status_and_sex_A92,Personal_status_and_sex_A93
0,6,1169,67,2,4,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
1,48,5951,22,1,2,2,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0
2,12,2096,49,1,2,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1
3,42,7882,45,1,2,4,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1
4,24,4870,53,2,3,4,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


## 3. Divide Training/Test Dataset

In [44]:
from sklearn.model_selection import train_test_split

In [45]:
train_x, test_x, train_y, test_y = \
    train_test_split( \
        final_dataset, dataset['Target'], test_size=0.2, random_state=42)

In [46]:
print('train_set', train_x.shape)
print('test_set', test_x.shape)

train_set (800, 40)
test_set (200, 40)


## 4. Use Statistical Model for Logistic Regression

In [47]:
import statsmodels.api as sm

In [48]:
logistic_model = sm.Logit(train_y, sm.add_constant(train_x)).fit()

Optimization terminated successfully.
         Current function value: 0.460159
         Iterations 7


## 5. Performance Measure

In [49]:
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [50]:
train_pred = pd.DataFrame({
    'probs': logistic_model.predict(sm.add_constant(train_x)),
    'class': train_y
})

train_pred['y_pred'] = 0
train_pred.loc[train_pred['probs'] > 0.5, 'y_pred'] = 1

# Test prediction
test_pred = pd.DataFrame({
    'probs': logistic_model.predict(sm.add_constant(test_x)),
    'class': test_y
})

test_pred['y_pred'] = 0
test_pred.loc[test_pred['probs'] > 0.5, 'y_pred'] = 1

In [53]:
print('\nTraining Confusion matrix:')
display(pd.crosstab(train_pred['y_pred'], train_pred['class'], rownames=['Predict'], colnames=['Actual'], margins=True))
print('\nTraining Accuracy: ', round(accuracy_score(train_pred['class'], train_pred['y_pred']), 4))
print('\nTraining classification report:\n', classification_report(train_pred['class'], train_pred['y_pred'], digits=4))


Training Confusion matrix:


Actual,0,1,All
Predict,,,
0,503,120,623
1,56,121,177
All,559,241,800



Training Accuracy:  0.78

Training classification report:
               precision    recall  f1-score   support

           0     0.8074    0.8998    0.8511       559
           1     0.6836    0.5021    0.5789       241

   micro avg     0.7800    0.7800    0.7800       800
   macro avg     0.7455    0.7009    0.7150       800
weighted avg     0.7701    0.7800    0.7691       800



In [54]:
print('Test Confusion matrix:')
display(pd.crosstab(test_pred['y_pred'], test_pred['class'], rownames=['Predict'], colnames=['Actual'], margins=True))
print('\nTest Accuracy: ', round(accuracy_score(test_pred['class'], test_pred['y_pred']), 4))
print('\nTest classification report:\n', classification_report(test_pred['class'], test_pred['y_pred'], digits=4))

Test Confusion matrix:


Actual,0,1,All
Predict,,,
0,123,27,150
1,18,32,50
All,141,59,200



Test Accuracy:  0.775

Test classification report:
               precision    recall  f1-score   support

           0     0.8200    0.8723    0.8454       141
           1     0.6400    0.5424    0.5872        59

   micro avg     0.7750    0.7750    0.7750       200
   macro avg     0.7300    0.7074    0.7163       200
weighted avg     0.7669    0.7750    0.7692       200



# 2. 후진제거법

In [55]:
unnecesarries = []

In [56]:
logistic_model.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                          Results: Logit
===================================================================================================
Model:                          Logit                       Pseudo R-squared:            0.248     
Dependent Variable:             Target                      AIC:                         818.2538  
Date:                           2019-04-24 21:13            BIC:                         1010.3229 
No. Observations:               800                         Log-Likelihood:              -368.13   
Df Model:                       40                          LL-Null:                     -489.54   
Df Residuals:                   759                         LLR p-value:                 7.2712e-31
Converged:                      1.0000                      Scale:                       1.0000    
No. Iterations:                 7.0000                                                             
---------------------------------------------------------------------------------------------------
                                                     Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
---------------------------------------------------------------------------------------------------
const                                               -3.6675   1.0471 -3.5026 0.0005 -5.7198 -1.6153
Duration_in_month                                    0.0230   0.0101  2.2759 0.0228  0.0032  0.0429
Credit_amount                                        0.0001   0.0000  2.4648 0.0137  0.0000  0.0002
Age_in_years                                        -0.0315   0.0104 -3.0316 0.0024 -0.0519 -0.0111
Number_of_existing_credits_at_this_bank              0.3461   0.2075  1.6682 0.0953 -0.0605  0.7528
Installment_rate_in_percentage_of_disposable_income  0.3172   0.0945  3.3582 0.0008  0.1321  0.5024
Present_residence_since                              0.0320   0.0948  0.3373 0.7359 -0.1538  0.2178
Status_of_existing_checking_account_A11              1.5522   0.2525  6.1468 0.0000  1.0573  2.0472
Status_of_existing_checking_account_A12              1.1125   0.2474  4.4969 0.0000  0.6276  1.5974
Status_of_existing_checking_account_A13              0.5329   0.4364  1.2211 0.2220 -0.3224  1.3881
Credit_history_A30                                   1.5371   0.4723  3.2545 0.0011  0.6114  2.4628
Credit_history_A31                                   1.4798   0.4815  3.0734 0.0021  0.5361  2.4235
Credit_history_A32                                   0.8741   0.2821  3.0988 0.0019  0.3212  1.4269
Credit_history_A33                                   0.5364   0.3691  1.4532 0.1462 -0.1871  1.2599
Savings_account_bonds_A61                            0.6851   0.2741  2.4997 0.0124  0.1479  1.2224
Savings_account_bonds_A62                            0.5970   0.3698  1.6144 0.1064 -0.1278  1.3217
Savings_account_bonds_A63                            0.2964   0.4700  0.6305 0.5284 -0.6249  1.2176
Savings_account_bonds_A64                           -0.2078   0.5857 -0.3547 0.7228 -1.3558  0.9402
Purpose_A40                                          0.6410   0.3599  1.7810 0.0749 -0.0644  1.3464
Purpose_A41                                         -0.9585   0.4654 -2.0593 0.0395 -1.8707 -0.0462
Purpose_A410                                        -0.6272   0.8378 -0.7486 0.4541 -2.2693  1.0149
Purpose_A42                                         -0.2957   0.3846 -0.7686 0.4421 -1.0496  0.4582
Purpose_A43                                         -0.3352   0.3618 -0.9263 0.3543 -1.0443  0.3740
Purpose_A44                                          0.1854   0.8089  0.2292 0.8187 -1.4000  1.7709
Purpose_A45                                          0.2867   0.6258  0.4581 0.6469 -0.9398  1.5131
Purpose_A46                                          0.9372   0.5224  1.7941 0.0728 -0.0867  1.9611
Purpose_A48                                         -1.2412   1.2458 -0.9964 0.3191 -3.6829  1

In [57]:
# Help 함수
def get_unrelated_cols(model, pvalue):
    cols = model.pvalues[model.pvalues >= pvalue].keys().tolist()
    print(len(cols))
    print(cols)
    
    return cols

In [58]:
unrelated_cols = get_unrelated_cols(logistic_model, 0.8187)

1
['Personal_status_and_sex_A92']


In [59]:
unnecesarries.append('Personal_status_and_sex_A92')

In [60]:
def get_max_vif(df, removal_cols):
    vifs = []
    cnames = df.drop(removal_cols, axis=1).columns.tolist()
    for i in range(len(cnames)):
        xvar = cnames[:]
        yvar = xvar.pop(i)
        model = sm.OLS(
            df.drop(removal_cols, axis=1)[yvar], 
            sm.add_constant(df.drop(removal_cols, axis=1)[xvar]))
        res = model.fit()
        vif = 1 / (1 - res.rsquared)
        vifs.append((yvar, round(vif, 3)))
    vifs = sorted(vifs, key=operator.itemgetter(1), reverse=True)
    return vifs

In [61]:
vifs = get_max_vif(train_x, unnecesarries)

In [62]:
vifs

[('Housing_A152', 5.838),
 ('Property_A123', 4.993),
 ('Property_A121', 4.77),
 ('Housing_A151', 4.625),
 ('Property_A122', 4.274),
 ('Purpose_A43', 3.161),
 ('Purpose_A40', 2.983),
 ('Purpose_A42', 2.607),
 ('Credit_amount', 2.404),
 ('Purpose_A41', 2.12),
 ('Credit_history_A32', 2.011),
 ('Duration_in_month', 1.974),
 ('Present_employment_since_A73', 1.922),
 ('Savings_account_bonds_A61', 1.879),
 ('Present_employment_since_A72', 1.837),
 ('Present_employment_since_A74', 1.618),
 ('Purpose_A46', 1.562),
 ('Number_of_existing_credits_at_this_bank', 1.556),
 ('Savings_account_bonds_A62', 1.478),
 ('Status_of_existing_checking_account_A11', 1.465),
 ('Age_in_years', 1.457),
 ('Credit_history_A31', 1.368),
 ('Status_of_existing_checking_account_A12', 1.363),
 ('Personal_status_and_sex_A93', 1.336),
 ('Savings_account_bonds_A63', 1.332),
 ('Present_residence_since', 1.33),
 ('Credit_history_A33', 1.325),
 ('Present_employment_since_A71', 1.314),
 ('Installment_rate_in_percentage_of_dispos

In [63]:
unnecesarries.append(vifs[0][0])

In [64]:
unnecesarries

['Personal_status_and_sex_A92', 'Housing_A152']

# C Statistics

In [65]:
def get_c_stat(iv_pred):
    noraml_test_df = iv_pred[iv_pred['class'] == 0][['class', 'probs']]
    spammer_test_df = iv_pred[iv_pred['class'] == 1][['class', 'probs']]

    noraml_test_df['key'] = 0
    spammer_test_df['key'] = 0

    cross_join_df = noraml_test_df.merge(spammer_test_df, how='outer', on='key').drop('key', axis=1)

    cross_join_df['concordance'] = cross_join_df['probs_x'] < cross_join_df['probs_y']
    cross_join_df['in_concordance'] = cross_join_df['probs_x'] > cross_join_df['probs_y']
    cross_join_df['tie'] = cross_join_df['probs_x'] == cross_join_df['probs_y']

    results = cross_join_df.agg({'concordance': np.sum, 'in_concordance': np.sum, 'tie': np.sum}) / len(cross_join_df)
    c_stat = 0.5 + (results['concordance'] - results['in_concordance']) / 2
    
    return c_stat

In [66]:
def get_aic_value(model):
    return -2 * model.llf + 2 * (len(model.params) - 1)

In [67]:
c_stat, aic = get_c_stat(train_pred), get_aic_value(logistic_model)
print('c_stat:', c_stat)
print('aic:', aic)
print('loglikehood:', logistic_model.llf)

c_stat: 0.8230390665013844
aic: 816.2537855406207
loglikehood: -368.12689277031035


In [68]:
c_stat, aic = get_c_stat(test_pred), get_aic_value(logistic_model)
print('c_stat:', c_stat)
print('aic:', aic)
print('loglikehood:', logistic_model.llf)

c_stat: 0.8067075369635773
aic: 816.2537855406207
loglikehood: -368.12689277031035
